# Differential Gene Regulatory Network Analysis

The first step is to import the required packages.

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("recount")
BiocManager::install("recount3")
BiocManager::install("limma")
BiocManager::install("DESeq2")
BiocManager::install("vsn")
BiocManager::install("biomaRt")
BiocManager::install("fgsea")
BiocManager::install("edgeR")

install.packages("visNetwork")
install.packages("ggplot2")
install.packages("dplyr")
install.packages("ggrepel")
install.packages("gridExtra")

In [ ]:
library('recount')    # To process gene expression
library('recount3')   # To process gene expression
library('limma')      # To average replicates
library('DESeq2')     # To do the mean-variance correction
library('vsn')        # To visualize mean-variance plots
library('biomaRt')    # To convert ensembl gene/protein ids to hugo (hgnc) symbols
library('visNetwork') # For network visualization 
library('fgsea')      # For gene set enrichment analysis 
library('ggplot2')    # To build the bubble plot for gene enrichment
library('dplyr')      # For data manipulation
library('edgeR')
library('ggrepel')    # Prevents overlapping text
library('gridExtra')

The second step is to import TCGA Human Pancreatic Adenocarcinoma and GTEx Pancreas data from Recount 3.

In [ ]:
rse_geneTumorDataOriginal <- recount3::create_rse_manual( project = "PAAD",
  project_home = "data_sources/tcga",
  organism = "human",
  annotation = "gencode_v26",
  type = "gene"
)

rse_geneNormalDataOriginal <- recount3::create_rse_manual(
  project = "PANCREAS",
  project_home = "data_sources/gtex",
  organism = "human",
  annotation = "gencode_v26",
  type = "gene"
)

The next step is to import a list of Human TFs and lists of X chromosome genes and Y chromosome genes.

In [ ]:
human_tf <- as.data.frame(read.table("http://humantfs.ccbr.utoronto.ca/download/v_1.01/TF_names_v_1.01.txt", header = FALSE, sep = "\n", dec = "."))
human_tf=human_tf[["V1"]]

Ygene <- as.vector(read.table("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&status=Approved&chr=Y&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&where=(gd_pub_chrom_map%20not%20like%20%27%25patch%25%27%20and%20gd_pub_chrom_map%20not%20like%20%27%25alternate%20reference%20locus%25%27)&submit=submit", header = TRUE, sep = "\n"))
Xgene <- as.vector(read.table("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&status=Approved&chr=X&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&where=(gd_pub_chrom_map%20not%20like%20%27%25patch%25%27%20and%20gd_pub_chrom_map%20not%20like%20%27%25alternate%20reference%20locus%25%27)&submit=submit", header = TRUE, sep = "\n"))

The "vsdcreator" function takes in a Random Summarized Experiment object, subsets the genes so that only genes of the specified gene type remain, removed the genes that have less than or equal to one count across all samples, and outputs meanSd plots and count matrix if desired. 

In [ ]:
vsdcreator <- function(rseobject, genetype, includePlots, includecountMat, isTumor){
  rse_gene <- rseobject
  rowDataRse=rowData(rse_gene)
  colDataRse=colData(rse_gene)
  
  geneIds = c()
  geneNames = c()
  geneEntrezIds = c()
  
  for(i in 1:dim(rowDataRse)[1]){
    if(rowDataRse@listData$gene_type[i][[1]][1] == genetype){
      geneIds       = c(geneIds,i)
      geneNames     = c(geneNames,rowDataRse@listData$gene_name[i][[1]][1])
      geneEntrezIds = c(geneEntrezIds, substr(rowDataRse@listData$gene_id[i],1,15))
    }
  }
  
  if(isTumor == TRUE){
    tumorIds=c()
    patientSymbol=c()
    
    for(i in 1:length(colDataRse@listData$tcga.gdc_cases.samples.sample_type)){
      if(colDataRse@listData$tcga.gdc_cases.samples.sample_type[i] %in% c("Primary Tumor","Metastatic") && colDataRse@listData$tcga.gdc_cases.project.name[i] == "Pancreatic Adenocarcinoma"){
        tumorIds = c(tumorIds,i)     
        patientSymbol = c(patientSymbol, colDataRse@listData$tcga.gdc_cases.samples.portions.analytes.aliquots.submitter_id[i])
      }
    }
    
    rse_gene=rse_gene[geneIds,tumorIds]
  }
  if(isTumor == FALSE){
    rse_gene=rse_gene[geneIds,]
  }
  
  
  countMat=SummarizedExperiment::assay(rse_gene, 1)
  
  indFilter = which(rowSums(countMat) <= 1)
  
  rse_gene=rse_gene[-indFilter,]
  
  rowDataRse=rowData(rse_gene)
  colDataRse=colData(rse_gene)
  
  countMat=SummarizedExperiment::assay(rse_gene, 1)
  meanSdPlotBeforeCorrection <- meanSdPlot(countMat)
  meanSdPlotBeforeCorrectionLogTrans <- meanSdPlot(log(countMat+1), ranks = FALSE)
  
  if (genetype == "miRNA"){
    vsd <- varianceStabilizingTransformation(countMat, blind=FALSE)
  } else{
  vsd <- vst(countMat, blind=FALSE)
  }
  
  meanSdPlot <- meanSdPlot(vsd)
  
  
  if(isTumor == TRUE){
    colnames(vsd)=patientSymbol
  }
  rownames(vsd)=geneNames[-indFilter]
  
  vsd=avearrays(t(vsd))
  vsd=as.data.frame(t(vsd))
  
  if(isTumor == TRUE){
    colnames(countMat)=patientSymbol
  }
  rownames(countMat)=geneNames[-indFilter]
  countMat=avearrays(t(countMat))
  countMat=as.data.frame(t(countMat))
  
  if (includePlots == FALSE & includecountMat == FALSE){
    return(list(rse = rse_gene, vsdoutput = vsd))
  }
  if (includePlots == TRUE & includecountMat == FALSE){
    return(list(rse = rse_gene,vsdoutput = vsd, meanSdPlotBeforeCorrectionoutput = meanSdPlotBeforeCorrection, meanSdPlotBeforeCorrectionLogTransoutput = meanSdPlotBeforeCorrectionLogTrans, meanSdPlotoutput = meanSdPlot))
  }
  
  if (includePlots == TRUE & includecountMat == TRUE){
    return(list(rse = rse_gene, vsdoutput = vsd, meanSdPlotBeforeCorrectionoutput = meanSdPlotBeforeCorrection, meanSdPlotBeforeCorrectionLogTransoutput = meanSdPlotBeforeCorrectionLogTrans, meanSdPlotoutput = meanSdPlot, countMatoutput = countMat))
  }
  
  if (includePlots == FALSE & includecountMat == TRUE){
    return(list(rse = rse_gene, vsdoutput = vsd, countMatoutput = countMat))
  }
}

The next step is to create TCGA Random Summarized Experiment objects. One has only protein coding genes, another has only lincRNA genes, and another only has miRNA genes. Count matrices with variance-stabilization transform applied are also created.

In [ ]:
vsdcreatorTumorOutput <- vsdcreator(rseobject = rse_geneTumorDataOriginal, genetype = "protein_coding", includePlots = TRUE, includecountMat = TRUE, isTumor = TRUE)

rse_geneTumorData <- vsdcreatorTumorOutput[[1]]
vsdTumorData <- vsdcreatorTumorOutput[[2]]

vsdcreatorTumorOutputlincRNA <- vsdcreator(rseobject = rse_geneTumorDataOriginal, genetype = "lincRNA", includePlots = TRUE, includecountMat = TRUE, isTumor = TRUE)

rse_geneTumorDatalincRNA <- vsdcreatorTumorOutputlincRNA[[1]]
vsdTumorDatalincRNA <- vsdcreatorTumorOutputlincRNA[[2]]

vsdcreatorTumorOutputmiRNA <- vsdcreator(rseobject = rse_geneTumorDataOriginal, genetype = "miRNA", includePlots = TRUE, includecountMat = TRUE, isTumor = TRUE)

rse_geneTumorDatamiRNA <- vsdcreatorTumorOutputmiRNA[[1]]
vsdTumorDatamiRNA <- vsdcreatorTumorOutputmiRNA[[2]]

The next step is to create GTEx Random Summarized Experiment objects. One has only protein coding genes, another has only lincRNA genes, and another only has miRNA genes. Count matrices with variance-stabilization transform applied are also created.

In [ ]:
vsdcreatorNormalOutput <- vsdcreator(rseobject = rse_geneNormalDataOriginal, genetype = "protein_coding", includePlots = TRUE, includecountMat = TRUE, isTumor = FALSE)

rse_geneNormalData <- vsdcreatorNormalOutput[[1]]
vsdNormalData <- vsdcreatorNormalOutput[[2]]

vsdcreatorNormalOutputlincRNA <- vsdcreator(rseobject = rse_geneNormalDataOriginal, genetype = "lincRNA", includePlots = TRUE, includecountMat = TRUE, isTumor = FALSE)

rse_geneNormalDatalincRNA <- vsdcreatorNormalOutputlincRNA[[1]]
vsdNormalDatalincRNA <- vsdcreatorNormalOutputlincRNA[[2]]

vsdcreatorNormalOutputmiRNA <- vsdcreator(rseobject = rse_geneNormalDataOriginal, genetype = "miRNA", includePlots = TRUE, includecountMat = TRUE, isTumor = FALSE)

rse_geneNormalDatamiRNA <- vsdcreatorNormalOutputmiRNA[[1]]
vsdNormalDatamiRNA <- vsdcreatorNormalOutputmiRNA[[2]]

The functions "tumorsexsubsetter" and "normalsexsubsetter" take a Random Summarized Experiment object with TCGA data and a Random Summarized Experiment object with GTEx data respectively and subsets the samples by sex ("M" stands for male and "F" stands for female).

In [ ]:
tumorsexsubsetter <- function(rseobject, sex){
  if (sex == "M"){
    tcgaidentifier = "male"
  }
  if (sex == "F"){
    tcgaidentifier = "female"
  }
  
  rse_gene <- rseobject
  colDataRse <- colData(rseobject)
  tumorIds = c()
  patientSymbol = c()
  
  for(i in 1:length(colDataRse@listData$tcga.gdc_cases.demographic.gender)){
    if((colDataRse@listData$tcga.gdc_cases.demographic.gender[i] %in% c(tcgaidentifier)) && (colDataRse@listData$tcga.gdc_cases.project.name[i] == "Pancreatic Adenocarcinoma")){
      tumorIds = c(tumorIds,i)
      patientSymbol = c(patientSymbol, colDataRse@listData$tcga.gdc_cases.samples.portions.analytes.aliquots.submitter_id[i])
    }
  }
  
  rse_gene=rse_gene[,tumorIds]
  colnames(rse_gene) = patientSymbol
  
  return(list(rse = rse_gene, Ids = tumorIds))
  
}

normalsexsubsetter <- function(rseobject, sex){
  if (sex == "M"){
    gtexidentifier = 1
  }
  if (sex == "F"){
    gtexidentifier = 2
  }
  
  rse_gene <- rseobject
  colDataRse <- colData(rseobject)
  normalIds = c()
  patientSymbol = c()
  
  for(i in 1:length(colDataRse@listData$gtex.sex)){
    if((colDataRse@listData$gtex.sex[i] == gtexidentifier)){
      normalIds = c(normalIds,i)     
      patientSymbol = c(patientSymbol, colDataRse@listData$subjid[i])
    }
  }
  rse_gene=rse_gene[,normalIds]
  colnames(rse_gene) = paste0(colnames(rse_gene), colnamesidentifier)
  
  return(list(rse = rse_gene, Ids = normalIds))
  
}

The function "sexsubsettervsd" subsets a count matrix with Variance Stabilization Transform applied so that it only contains the samples in the Random Summarized Experiment Object.

In [ ]:
sexsubsettervsd <- function(rse_gene, vsd){
  list <- rse_gene@colData@rownames
  for (i in 1:length(list)){
    list[i] <- substr(list[i], 1, nchar(list[i]))
  }
  vsdoutput <- vsd[,list]

  return(vsdoutput)
  
}

In the next step we create the objects "vsdMaleTumor," "rse_geneMaleTumormiRNA," "rse_geneMaleTumorlincRNA," "vsdFeMaleTumor," "rse_geneFeMaleTumormiRNA," and "rse_geneFeMaleTumorlincRNA."

In [ ]:
# protein-coding genes
tumorsexsubsetterTumorMaleOutput <-tumorsexsubsetter(rse_geneTumorData, "M")

rse_geneMaleTumor <- tumorsexsubsetterTumorMaleOutput[[1]]

vsdMaleTumor <- sexsubsettervsd(rse_geneMaleTumor, vsdTumorData)

tumorsexsubsetterTumorFeMaleOutput <-tumorsexsubsetter(rse_geneTumorData, "F")

rse_geneFeMaleTumor <- tumorsexsubsetterTumorFeMaleOutput[[1]]

vsdFeMaleTumor <- sexsubsettervsd(rse_geneFeMaleTumor, vsdTumorData)

# miRNA genes

tumorsexsubsetterTumorMaleOutputmiRNA <-tumorsexsubsetter(rse_geneTumorDatamiRNA, "M")

rse_geneMaleTumormiRNA <- tumorsexsubsetterTumorMaleOutputmiRNA[[1]]

vsdMaleTumormiRNA <- sexsubsettervsd(rse_geneMaleTumormiRNA, vsdTumorDatamiRNA)

tumorsexsubsetterTumorFeMaleOutputmiRNA <-tumorsexsubsetter(rse_geneTumorDatamiRNA, "F")

rse_geneFeMaleTumormiRNA <- tumorsexsubsetterTumorFeMaleOutputmiRNA[[1]]

vsdFeMaleTumormiRNA <- sexsubsettervsd(rse_geneFeMaleTumormiRNA, vsdTumorDatamiRNA)

# lincRNA genes

tumorsexsubsetterTumorMaleOutputlincRNA <-tumorsexsubsetter(rse_geneTumorDatalincRNA, "M")

rse_geneMaleTumorlincRNA <- tumorsexsubsetterTumorMaleOutputlincRNA[[1]]

vsdMaleTumorlincRNA <- sexsubsettervsd(rse_geneMaleTumorlincRNA, vsdTumorDatalincRNA)

tumorsexsubsetterTumorFeMaleOutputlincRNA <-tumorsexsubsetter(rse_geneTumorDatalincRNA, "F")

rse_geneFeMaleTumorlincRNA <- tumorsexsubsetterTumorFeMaleOutputlincRNA[[1]]

vsdFeMaleTumorlincRNA <- sexsubsettervsd(rse_geneFeMaleTumorlincRNA, vsdTumorDatalincRNA)

In the next step we create the objects "vsdMaleNormal," "rse_geneMaleNormalmiRNA," "rse_geneMaleNormallincRNA," "vsdFeMaleNormal," "rse_geneFeMaleNormalmiRNA," and "rse_geneFeMaleNormallincRNA."

In [ ]:
# protein-coding genes

normalsexsubsetterNormalMaleOutput <- normalsexsubsetter(rse_geneNormalData, "M")

rse_geneMaleNormal <- normalsexsubsetterNormalMaleOutput[[1]]
MaleNormalIds <- normalsexsubsetterNormalMaleOutput[[2]]

vsdMaleNormal <- sexsubsettervsd(rse_geneMaleNormal, vsdNormalData)

normalsexsubsetterNormalFeMaleOutput <-normalsexsubsetter(rse_geneNormalData, "F")

rse_geneFeMaleNormal <- normalsexsubsetterNormalFeMaleOutput[[1]]
FeMaleNormalIds <- normalsexsubsetterNormalFeMaleOutput[[2]]

vsdFeMaleNormal <- sexsubsettervsd(rse_geneFeMaleNormal, vsdNormalData)

# miRNA genes

normalsexsubsetterNormalMaleOutputmiRNA <-normalsexsubsetter(rse_geneNormalDatamiRNA, "M")

rse_geneMaleNormalmiRNA <- normalsexsubsetterNormalMaleOutputmiRNA[[1]]

vsdMaleNormalmiRNA <- sexsubsettervsd(rse_geneMaleNormalmiRNA, vsdNormalDatamiRNA)

normalsexsubsetterNormalFeMaleOutputmiRNA <-normalsexsubsetter(rse_geneNormalDatamiRNA, "F")

rse_geneFeMaleNormalmiRNA <- normalsexsubsetterNormalFeMaleOutputmiRNA[[1]]

vsdFeMaleNormalmiRNA <- sexsubsettervsd(rse_geneFeMaleNormalmiRNA, vsdNormalDatamiRNA)

# lincRNA genes

normalsexsubsetterNormalMaleOutputlincRNA <-normalsexsubsetter(rse_geneNormalDatalincRNA, "M")

rse_geneMaleNormallincRNA <- normalsexsubsetterNormalMaleOutputlincRNA[[1]]

vsdMaleNormallincRNA <- sexsubsettervsd(rse_geneMaleNormallincRNA, vsdNormalDatalincRNA)

normalsexsubsetterNormalFeMaleOutputlincRNA <-normalsexsubsetter(rse_geneNormalDatalincRNA, "F")

rse_geneFeMaleNormallincRNA <- normalsexsubsetterNormalFeMaleOutputlincRNA[[1]]

vsdFeMaleNormallincRNA <- sexsubsettervsd(rse_geneFeMaleNormallincRNA, vsdNormalDatalincRNA)


In the next step we create a visnetwork graph displaying the results of a differential gene regulatory network analysis between gene regulatory networks created using TCGA samples and GTEx Samples.

In [ ]:
NormalAllMatrix <- as.data.frame(t(rbind(vsdcreator(rseobject = rse_geneNormalDataOriginal, genetype = "protein_coding", includePlots = FALSE, includecountMat = FALSE, isTumor = FALSE)[[2]], vsdcreator(rseobject = rse_geneNormalDataOriginal, genetype = "lincRNA", includePlots = FALSE, includecountMat = FALSE, isTumor = FALSE)[[2]], vsdcreator(rseobject = rse_geneNormalDataOriginal, genetype = "miRNA", includePlots = FALSE, includecountMat = FALSE, isTumor = FALSE)[[2]])))

TumorAllMatrix <- as.data.frame(t(rbind(vsdcreator(rseobject = rse_geneTumorDataOriginal, genetype = "protein_coding", includePlots = FALSE, includecountMat = FALSE, isTumor = TRUE)[[2]], vsdcreator(rseobject = rse_geneTumorDataOriginal, genetype = "lincRNA", includePlots = FALSE, includecountMat = FALSE, isTumor = TRUE)[[2]], vsdcreator(rseobject = rse_geneTumorDataOriginal, genetype = "miRNA", includePlots = FALSE, includecountMat = FALSE, isTumor = TRUE)[[2]])))

ListNATAComp <- intersect(colnames(NormalAllMatrix), colnames(TumorAllMatrix))

CorNormalAllMatrix <- as.data.frame(cor(NormalAllMatrix[,ListNATAComp]))
CorTumorAllMatrix <- as.data.frame(cor(TumorAllMatrix[,ListNATAComp]))


CorNormalAllMatrix <- CorNormalAllMatrix %>% select(as.vector(intersect(ListNATAComp, rownames(vsdNormalData))))
CorNormalAllMatrix <- CorNormalAllMatrix %>% select(-as.vector(intersect(intersect(ListNATAComp, rownames(vsdNormalData)), human_tf)))
CorNormalAllMatrix <- as.data.frame(subset(CorNormalAllMatrix, !(rownames(CorNormalAllMatrix) %in% colnames(CorNormalAllMatrix))))

CorTumorAllMatrix <- CorTumorAllMatrix %>% select(as.vector(intersect(ListNATAComp, rownames(vsdTumorData))))
CorTumorAllMatrix <- CorTumorAllMatrix %>% select(-as.vector(intersect(intersect(ListNATAComp, rownames(vsdTumorData)), human_tf)))
CorTumorAllMatrix <- as.data.frame(subset(CorTumorAllMatrix, !(rownames(CorTumorAllMatrix) %in% colnames(CorTumorAllMatrix))))

AllComparisonMatrix <- CorTumorAllMatrix - CorNormalAllMatrix
AllComparisonMatrixrownames = rownames(AllComparisonMatrix)

num_edges <- 100

edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

sort_mat = order(as.matrix(abs(AllComparisonMatrix)), decreasing = TRUE)
edges$value  = as.matrix(AllComparisonMatrix)[sort_mat[1:num_edges]]

pcgeneIdsTop = (sort_mat[1:num_edges] %/% dim(AllComparisonMatrix)[1]) + 1
rnaIdsTop = sort_mat[1:num_edges] %% dim(AllComparisonMatrix)[1]
nrnas = dim(AllComparisonMatrix)[1]
rnaIdsTop[rnaIdsTop == 0] = nrnas

edges$to = colnames(AllComparisonMatrix)[pcgeneIdsTop]
edges$from = rownames(AllComparisonMatrix)[rnaIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
#nodes$group = ifelse(nodes$id %in% edges$from, "miRNA/lincRNA", "Protein-coding Gene")
idmirna <- c()
idlncrna <- c()
idtfs <- c()
mirnavector <- as.vector(union(rownames(vsdNormalDatamiRNA),rownames(vsdTumorDatamiRNA)))
lncrnavector <- as.vector(union(rownames(vsdNormalDatalincRNA),rownames(vsdTumorDatalincRNA)))

for(i in 1:dim(nodes)[1]){
  if(nodes[i,1] %in% human_tf){
    idtfs = c(idtfs, i)
  }
  if(nodes[i,1] %in% mirnavector){
    idmirna = c(idmirna, i)
  }
  if(nodes[i,1] %in% lncrnavector){
    idlncrna = c(idlncrna, i)
  }
}
nodes$group = 'Protein-coding Gene'
nodes$group[idmirna] = 'miRNA'
nodes$group[idlncrna] = 'lncRNA'
nodes$group[idtfs] = 'tfs'

net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "tfs", shape = "triangle",
                 color = list(background = "green", border="green"))
net <- visGroups(net, groupname = "lncRNA", shape = "triangle",
                 color = list(background = "pink", border="pink"))
net <- visGroups(net, groupname = "miRNA", shape = "triangle",
                 color = list(background = "orange", border="orange"))
net <- visGroups(net, groupname = "Protein-coding Gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)

In the next step we create a visnetwork graph displaying the results of a differential gene regulatory network analysis between gene regulatory networks created using male TCGA samples and male GTEx Samples.

In [ ]:
NormalMaleMatrix <- as.data.frame(t(rbind(vsdMaleNormal, vsdMaleNormalmiRNA, vsdMaleNormallincRNA)))

TumorMaleMatrix <- as.data.frame(t(rbind(vsdMaleTumor, vsdMaleTumormiRNA, vsdMaleTumorlincRNA)))

ListNMTMComp <- intersect(colnames(NormalMaleMatrix), colnames(TumorMaleMatrix))

CorNormalMaleMatrix <- as.data.frame(cor(NormalMaleMatrix[,ListNMTMComp]))
CorTumorMaleMatrix <- as.data.frame(cor(TumorMaleMatrix[,ListNMTMComp]))


CorNormalMaleMatrix <- CorNormalMaleMatrix %>% select(as.vector(intersect(ListNMTMComp, rownames(vsdMaleNormal))))
CorNormalMaleMatrix <- CorNormalMaleMatrix %>% select(-as.vector(intersect(intersect(ListNMTMComp, rownames(vsdMaleNormal)), human_tf)))
CorNormalMaleMatrix <- as.data.frame(subset(CorNormalMaleMatrix, !(rownames(CorNormalMaleMatrix) %in% colnames(CorNormalMaleMatrix))))

CorTumorMaleMatrix <- CorTumorMaleMatrix %>% select(as.vector(intersect(ListNMTMComp, rownames(vsdMaleTumor))))
CorTumorMaleMatrix <- CorTumorMaleMatrix %>% select(-as.vector(intersect(intersect(ListNMTMComp, rownames(vsdMaleTumor)), human_tf)))
CorTumorMaleMatrix <- as.data.frame(subset(CorTumorMaleMatrix, !(rownames(CorTumorMaleMatrix) %in% colnames(CorTumorMaleMatrix))))

MaleComparisonMatrix <- CorTumorMaleMatrix - CorNormalMaleMatrix
MaleComparisonMatrixrownames = rownames(MaleComparisonMatrix)

num_edges <- 100

edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

sort_mat = order(as.matrix(abs(MaleComparisonMatrix)), decreasing = TRUE)
edges$value  = as.matrix(MaleComparisonMatrix)[sort_mat[1:num_edges]]

pcgeneIdsTop = (sort_mat[1:num_edges] %/% dim(MaleComparisonMatrix)[1]) + 1
rnaIdsTop = sort_mat[1:num_edges] %% dim(MaleComparisonMatrix)[1]
nrnas = dim(MaleComparisonMatrix)[1]
rnaIdsTop[rnaIdsTop == 0] = nrnas

edges$to = colnames(MaleComparisonMatrix)[pcgeneIdsTop]
edges$from = rownames(MaleComparisonMatrix)[rnaIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
#nodes$group = ifelse(nodes$id %in% edges$from, "miRNA/lincRNA", "Protein-coding Gene")
idmirna <- c()
idlncrna <- c()
idtfs <- c()
mirnavector <- as.vector(union(rownames(vsdMaleNormalmiRNA),rownames(vsdMaleTumormiRNA)))
lncrnavector <- as.vector(union(rownames(vsdMaleNormallincRNA),rownames(vsdMaleTumorlincRNA)))

for(i in 1:dim(nodes)[1]){
  if(nodes[i,1] %in% human_tf){
    idtfs = c(idtfs, i)
  }
  if(nodes[i,1] %in% mirnavector){
    idmirna = c(idmirna, i)
  }
  if(nodes[i,1] %in% lncrnavector){
    idlncrna = c(idlncrna, i)
  }
}
nodes$group = 'Protein-coding Gene'
nodes$group[idmirna] = 'miRNA'
nodes$group[idlncrna] = 'lncRNA'
nodes$group[idtfs] = 'tfs'

net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "tfs", shape = "triangle",
                 color = list(background = "green", border="green"))
net <- visGroups(net, groupname = "lncRNA", shape = "triangle",
                 color = list(background = "pink", border="pink"))
net <- visGroups(net, groupname = "miRNA", shape = "triangle",
                 color = list(background = "orange", border="orange"))
net <- visGroups(net, groupname = "Protein-coding Gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)

In the next step we create a visnetwork graph displaying the results of a differential gene regulatory network analysis between gene regulatory networks created using female TCGA samples and female GTEx Samples.

In [ ]:
NormalFeMaleMatrix <- as.data.frame(t(rbind(vsdFeMaleNormal, vsdFeMaleNormalmiRNA, vsdFeMaleNormallincRNA)))

TumorFeMaleMatrix <- as.data.frame(t(rbind(vsdFeMaleTumor, vsdFeMaleTumormiRNA, vsdFeMaleTumorlincRNA)))

ListNFTFComp <- intersect(colnames(NormalFeMaleMatrix), colnames(TumorFeMaleMatrix))

CorNormalFeMaleMatrix <- as.data.frame(cor(NormalFeMaleMatrix[,ListNFTFComp]))
CorTumorFeMaleMatrix <- as.data.frame(cor(TumorFeMaleMatrix[,ListNFTFComp]))

CorNormalFeMaleMatrix <- CorNormalFeMaleMatrix %>% select(as.vector(intersect(ListNFTFComp, rownames(vsdFeMaleNormal))))
CorNormalFeMaleMatrix <- CorNormalFeMaleMatrix %>% select(-as.vector(intersect(intersect(ListNFTFComp, rownames(vsdFeMaleNormal)), human_tf)))
CorNormalFeMaleMatrix <- as.data.frame(subset(CorNormalFeMaleMatrix, !(rownames(CorNormalFeMaleMatrix) %in% colnames(CorNormalFeMaleMatrix))))

CorTumorFeMaleMatrix <- CorTumorFeMaleMatrix %>% select(as.vector(intersect(ListNFTFComp, rownames(vsdFeMaleTumor))))
CorTumorFeMaleMatrix <- CorTumorFeMaleMatrix %>% select(-as.vector(intersect(intersect(ListNFTFComp, rownames(vsdFeMaleTumor)), human_tf)))
CorTumorFeMaleMatrix <- as.data.frame(subset(CorTumorFeMaleMatrix, !(rownames(CorTumorFeMaleMatrix) %in% colnames(CorTumorFeMaleMatrix))))

FeMaleComparisonMatrix <- CorTumorFeMaleMatrix - CorNormalFeMaleMatrix
FeMaleComparisonMatrixrownames = rownames(FeMaleComparisonMatrix)

num_edges <- 100

edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

sort_mat = order(as.matrix(abs(FeMaleComparisonMatrix)), decreasing = TRUE)
edges$value  = as.matrix(FeMaleComparisonMatrix)[sort_mat[1:num_edges]]

pcgeneIdsTop = (sort_mat[1:num_edges] %/% dim(FeMaleComparisonMatrix)[1]) + 1
rnaIdsTop = sort_mat[1:num_edges] %% dim(FeMaleComparisonMatrix)[1]
nrnas = dim(FeMaleComparisonMatrix)[1]
rnaIdsTop[rnaIdsTop == 0] = nrnas

edges$to = colnames(FeMaleComparisonMatrix)[pcgeneIdsTop]
edges$from = rownames(FeMaleComparisonMatrix)[rnaIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
#nodes$group = ifelse(nodes$id %in% edges$from, "miRNA/lincRNA", "Protein-coding Gene")
idmirna <- c()
idlncrna <- c()
idtfs <- c()
mirnavector <- as.vector(union(rownames(vsdFeMaleNormalmiRNA),rownames(vsdFeMaleTumormiRNA)))
lncrnavector <- as.vector(union(rownames(vsdFeMaleNormallincRNA),rownames(vsdFeMaleTumorlincRNA)))

for(i in 1:dim(nodes)[1]){
  if(nodes[i,1] %in% human_tf){
    idtfs = c(idtfs, i)
  }
  if(nodes[i,1] %in% mirnavector){
    idmirna = c(idmirna, i)
  }
  if(nodes[i,1] %in% lncrnavector){
    idlncrna = c(idlncrna, i)
  }
}
nodes$group = 'Protein-coding Gene'
nodes$group[idmirna] = 'miRNA'
nodes$group[idlncrna] = 'lncRNA'
nodes$group[idtfs] = 'tfs'

net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "tfs", shape = "triangle",
                 color = list(background = "green", border="green"))
net <- visGroups(net, groupname = "lncRNA", shape = "triangle",
                 color = list(background = "pink", border="pink"))
net <- visGroups(net, groupname = "miRNA", shape = "triangle",
                 color = list(background = "orange", border="orange"))
net <- visGroups(net, groupname = "Protein-coding Gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)

In the next step we create a visnetwork graph displaying the results of a differential gene regulatory network analysis between a gene regulatory network created using Male GTEx samples and Female GTEx samples. Note that X and Y chromosome genes have been removed.

In [ ]:
NormalMaleMatrix <- as.data.frame(t(rbind(vsdMaleNormal, vsdMaleNormalmiRNA, vsdMaleNormallincRNA)))

NormalFeMaleMatrix <- as.data.frame(t(rbind(vsdFeMaleNormal, vsdFeMaleNormalmiRNA, vsdFeMaleNormallincRNA)))

ListNMNFComp <- intersect(colnames(NormalMaleMatrix), colnames(NormalFeMaleMatrix))
ListNMNFComp <- ListTMTFComp[ListTMTFComp %in% Xgene]
ListNMNFComp <- ListTMTFComp[ListTMTFComp %in% Ygene]

CorNormalMaleMatrix <- as.data.frame(cor(NormalMaleMatrix[,ListNMNFComp]))
CorNormalFeMaleMatrix <- as.data.frame(cor(NormalFeMaleMatrix[,ListNMNFComp]))

CorNormalMaleMatrix <- CorNormalMaleMatrix %>% select(as.vector(intersect(ListNMNFComp, rownames(vsdMaleNormal))))
CorNormalMaleMatrix <- CorNormalMaleMatrix %>% select(-as.vector(intersect(intersect(ListNMNFComp, rownames(vsdMaleNormal)), human_tf)))
CorNormalMaleMatrix <- as.data.frame(subset(CorNormalMaleMatrix, !(rownames(CorNormalMaleMatrix) %in% colnames(CorNormalMaleMatrix))))

CorNormalFeMaleMatrix <- CorNormalFeMaleMatrix %>% select(as.vector(intersect(ListNMNFComp, rownames(vsdFeMaleNormal))))
CorNormalFeMaleMatrix <- CorNormalFeMaleMatrix %>% select(-as.vector(intersect(intersect(ListNMNFComp, rownames(vsdFeMaleNormal)), human_tf)))
CorNormalFeMaleMatrix <- as.data.frame(subset(CorNormalFeMaleMatrix, !(rownames(CorNormalFeMaleMatrix) %in% colnames(CorNormalFeMaleMatrix))))


NormalComparisonMatrix <- CorNormalMaleMatrix - CorNormalFeMaleMatrix
NormalComparisonMatrixrownames = rownames(NormalComparisonMatrix)

num_edges <- 100

edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

sort_mat = order(as.matrix(abs(NormalComparisonMatrix)), decreasing = TRUE)
edges$value  = as.matrix(NormalComparisonMatrix)[sort_mat[1:num_edges]]

pcgeneIdsTop = (sort_mat[1:num_edges] %/% dim(NormalComparisonMatrix)[1]) + 1
rnaIdsTop = sort_mat[1:num_edges] %% dim(NormalComparisonMatrix)[1]
nrnas = dim(NormalComparisonMatrix)[1]
rnaIdsTop[rnaIdsTop == 0] = nrnas

edges$to = colnames(NormalComparisonMatrix)[pcgeneIdsTop]
edges$from = rownames(NormalComparisonMatrix)[rnaIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
#nodes$group = ifelse(nodes$id %in% edges$from, "miRNA/lincRNA", "Protein-coding Gene")
idmirna <- c()
idlncrna <- c()
idtfs <- c()
mirnavector <- as.vector(rownames(vsdNormalDatamiRNA))
lncrnavector <- as.vector(rownames(vsdNormalDatamiRNA))

for(i in 1:dim(nodes)[1]){
  if(nodes[i,1] %in% human_tf){
    idtfs = c(idtfs, i)
  }
  if(nodes[i,1] %in% mirnavector){
    idmirna = c(idmirna, i)
  }
  if(nodes[i,1] %in% lncrnavector){
    idlncrna = c(idlncrna, i)
  }
}
nodes$group = 'Protein-coding Gene'
nodes$group[idmirna] = 'miRNA'
nodes$group[idlncrna] = 'lncRNA'
nodes$group[idtfs] = 'tfs'

net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "tfs", shape = "triangle",
                 color = list(background = "green", border="green"))
net <- visGroups(net, groupname = "lncRNA", shape = "triangle",
                 color = list(background = "pink", border="pink"))
net <- visGroups(net, groupname = "miRNA", shape = "triangle",
                 color = list(background = "orange", border="orange"))
net <- visGroups(net, groupname = "Protein-coding Gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)

In the next step we create a visnetwork graph displaying the results of a differential gene regulatory network analysis between a gene regulatory network created using Male TCGA samples and Female TCGA samples. Note that X and Y chromosome genes have been removed.

In [ ]:

TumorMaleMatrix <- as.data.frame(t(rbind(vsdMaleTumor, vsdMaleTumormiRNA, vsdMaleTumorlincRNA)))

TumorFeMaleMatrix <- as.data.frame(t(rbind(vsdFeMaleTumor, vsdFeMaleTumormiRNA, vsdFeMaleTumorlincRNA)))

ListTMTFComp <- intersect(colnames(TumorMaleMatrix), colnames(TumorFeMaleMatrix))
ListTMTFComp <- ListTMTFComp[ListTMTFComp %in% Xgene]
ListTMTFComp <- ListTMTFComp[ListTMTFComp %in% Ygene]

CorTumorMaleMatrix <- as.data.frame(cor(TumorMaleMatrix[,ListTMTFComp]))
CorTumorFeMaleMatrix <- as.data.frame(cor(TumorFeMaleMatrix[,ListTMTFComp]))

CorTumorMaleMatrix <- CorTumorMaleMatrix %>% select(as.vector(intersect(ListTMTFComp, rownames(vsdMaleTumor))))
CorTumorMaleMatrix <- CorTumorMaleMatrix %>% select(-as.vector(intersect(intersect(ListTMTFComp, rownames(vsdMaleTumor)), human_tf)))
CorTumorMaleMatrix <- as.data.frame(subset(CorTumorMaleMatrix, !(rownames(CorTumorMaleMatrix) %in% colnames(CorTumorMaleMatrix))))

CorTumorFeMaleMatrix <- CorTumorFeMaleMatrix %>% select(as.vector(intersect(ListTMTFComp, rownames(vsdFeMaleTumor))))
CorTumorFeMaleMatrix <- CorTumorFeMaleMatrix %>% select(-as.vector(intersect(intersect(ListTMTFComp, rownames(vsdFeMaleTumor)), human_tf)))
CorTumorFeMaleMatrix <- as.data.frame(subset(CorTumorFeMaleMatrix, !(rownames(CorTumorFeMaleMatrix) %in% colnames(CorTumorFeMaleMatrix))))


TumorComparisonMatrix <- CorTumorMaleMatrix - CorTumorFeMaleMatrix
TumorComparisonMatrixrownames = rownames(TumorComparisonMatrix)

num_edges <- 100

edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

sort_mat = order(as.matrix(abs(TumorComparisonMatrix)), decreasing = TRUE)
edges$value  = as.matrix(TumorComparisonMatrix)[sort_mat[1:num_edges]]

pcgeneIdsTop = (sort_mat[1:num_edges] %/% dim(TumorComparisonMatrix)[1]) + 1
rnaIdsTop = sort_mat[1:num_edges] %% dim(TumorComparisonMatrix)[1]
nrnas = dim(TumorComparisonMatrix)[1]
rnaIdsTop[rnaIdsTop == 0] = nrnas

edges$to = colnames(TumorComparisonMatrix)[pcgeneIdsTop]
edges$from = rownames(TumorComparisonMatrix)[rnaIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
#nodes$group = ifelse(nodes$id %in% edges$from, "miRNA/lincRNA", "Protein-coding Gene")
idmirna <- c()
idlncrna <- c()
idtfs <- c()
mirnavector <- as.vector(rownames(vsdTumorDatamiRNA))
lncrnavector <- as.vector(rownames(vsdTumorDatamiRNA))

for(i in 1:dim(nodes)[1]){
  if(nodes[i,1] %in% human_tf){
    idtfs = c(idtfs, i)
  }
  if(nodes[i,1] %in% mirnavector){
    idmirna = c(idmirna, i)
  }
  if(nodes[i,1] %in% lncrnavector){
    idlncrna = c(idlncrna, i)
  }
}
nodes$group = 'Protein-coding Gene'
nodes$group[idmirna] = 'miRNA'
nodes$group[idlncrna] = 'lncRNA'
nodes$group[idtfs] = 'tfs'

net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "tfs", shape = "triangle",
                 color = list(background = "green", border="green"))
net <- visGroups(net, groupname = "lncRNA", shape = "triangle",
                 color = list(background = "pink", border="pink"))
net <- visGroups(net, groupname = "miRNA", shape = "triangle",
                 color = list(background = "orange", border="orange"))
net <- visGroups(net, groupname = "Protein-coding Gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)